In [ ]:
import os
import glob
import SimpleITK as sitk
import numpy as np
from einops import rearrange

In [ ]:
# 1. Obtener los IDs de caso y los slices disponibles a partir de los archivos en la carpeta CTP
ctp_folder = "/data/dev/perfu-net/data/train/CTP"
ctp_files = glob.glob(os.path.join(ctp_folder, "case_*.npy"))
ctp_slices = {}  # Diccionario: key = case_id, value = set de slices (ej: "01", "02", ...)

for file in ctp_files:
    base = os.path.basename(file)  # ejemplo: "case_12_04.npy" o "case_1_04.npy"
    parts = base.split("_")
    if len(parts) >= 3:
        case_id = parts[1].zfill(2)  # Convertir a dos dígitos (ej: "1" -> "01")
        slice_num = os.path.splitext(parts[2])[0]  # "04.npy" -> "04"
        if case_id not in ctp_slices:
            ctp_slices[case_id] = set()
        ctp_slices[case_id].add(slice_num)

print("Casos presentes en CTP:", list(ctp_slices.keys()))

# 2. Buscar todas las máscaras en TRAINING que tengan "OT" en su path
training_path = "/data/ISLES-2018/TRAINING"
mask_files = glob.glob(os.path.join(training_path, "case_*", "**", "*OT*.nii"), recursive=True)

# Carpeta de salida para las máscaras individuales
mask_output_folder = "/data/dev/perfu-net/data/train/MASK"
os.makedirs(mask_output_folder, exist_ok=True)

# Carpeta de salida para la máscara completa
complete_mask_output_folder = "/data/dev/perfu-net/data/train/COMPLETE_MASK"
os.makedirs(complete_mask_output_folder, exist_ok=True)

# 3. Procesar cada archivo de máscara encontrado
for mask_file in mask_files:
    # Extraer el ID de caso a partir de la ruta (se asume carpeta con formato "case_X" o "case_XX")
    parts = mask_file.split(os.sep)
    case_name = None
    for part in parts:
        if part.startswith("case_"):
            case_name = part  # Ej: "case_12" o "case_1"
            break

    if case_name is None:
        print(f"No se encontró el ID de caso en {mask_file}. Se omite.")
        continue

    # Extraer el número de caso y convertirlo a dos dígitos
    patient_id_raw = case_name.split("_")[1]
    patient_id = patient_id_raw.zfill(2)
    
    # Solo procesar si el caso también está en CTP (con slices disponibles)
    if patient_id not in ctp_slices:
        print(f"Paciente {patient_id} no está en CTP. Se omite {mask_file}.")
        continue

    print(f"Procesando máscara para paciente {patient_id}: {mask_file}")
    
    # 3.1. Cargar la máscara usando SimpleITK y convertir a array NumPy
    mask_image = sitk.ReadImage(mask_file)
    mask_array = sitk.GetArrayFromImage(mask_image)
    # Reorganizamos el array a (h, w, d) para que z=0 corresponda a slice "01"
    mask_array = rearrange(mask_array, 'd h w -> h w d')
    
    # Obtener el conjunto de slices disponibles para este paciente en CTP
    available_slices = ctp_slices[patient_id]
    
    # Lista para acumular las slices válidas (las que están en CTP)
    valid_slices = []
    
    # 3.2. Dividir la máscara por cada slice (componente z)
    num_slices = mask_array.shape[2]
    for z in range(num_slices):
        # Formatear el número de slice (z=0 -> "01")
        slice_num = str(z+1).zfill(2)
        if slice_num not in available_slices:
            print(f"Slice {slice_num} no está en CTP para paciente {patient_id}. Se omite.")
            continue
        slice_mask = mask_array[:, :, z]
        valid_slices.append(slice_mask)
        # Guardar el slice individualmente en MASK
        out_filename = f"case_{patient_id}_{slice_num}.npy"
        out_path = os.path.join(mask_output_folder, out_filename)
        np.save(out_path, slice_mask)
        print(f"Guardado slice z={z} (archivo {out_filename}) en {out_path}")
    
    # 3.3. Guardar la máscara completa solo con los slices presentes en CTP
    if valid_slices:
        # Apilar las slices válidas a lo largo del eje z para formar el volumen
        complete_mask = np.stack(valid_slices, axis=0)
        complete_filename = f"case_{patient_id}.npy"
        complete_path = os.path.join(complete_mask_output_folder, complete_filename)
        np.save(complete_path, complete_mask)
        print(f"Guardada máscara completa para paciente {patient_id} con slices de CTP en {complete_path}")
    else:
        print(f"Paciente {patient_id}: no se encontraron slices válidos en CTP, no se guarda máscara completa.")

In [ ]:
# Definir las rutas a las carpetas
ctp_folder = "/data/dev/perfu-net-1/data/train/CTP"
mask_folder = "/data/dev/perfu-net-1/data/train/MASK"

# Obtener la lista de archivos en cada carpeta con el patrón "case_*.npy"
ctp_files = sorted([os.path.basename(f) for f in glob.glob(os.path.join(ctp_folder, "case_*.npy"))])
mask_files = sorted([os.path.basename(f) for f in glob.glob(os.path.join(mask_folder, "case_*.npy"))])

# Imprimir la lista de archivos de cada carpeta
print("Archivos en CTP:")
for f in ctp_files:
    print(f)
    
print("\nArchivos en MASK:")
for f in mask_files:
    print(f)

# Convertir las listas en conjuntos para facilitar la comparación
ctp_set = set(ctp_files)
mask_set = set(mask_files)

# Calcular diferencias
ctp_not_in_mask = sorted(ctp_set - mask_set)
mask_not_in_ctp = sorted(mask_set - ctp_set)

print("\nArchivos en CTP que NO están en MASK:")
for f in ctp_not_in_mask:
    print(f)

print("\nArchivos en MASK que NO están en CTP:")
for f in mask_not_in_ctp:
    print(f)
